In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, jaccard_score, matthews_corrcoef, roc_curve, auc

In [2]:
def load_images_from_dir(dir_path):
    images = []
    labels = []
    for class_dir in os.listdir(dir_path):
        class_path = os.path.join(dir_path, class_dir)
        label = 1 if class_dir.lower() == 'fire' else 0
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = load_img(img_path, target_size=(150, 150))
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(label)
    return np.array(images), np.array(labels)

In [3]:
original_dir = '/kaggle/input/forest-fire-classification-dataset/ForestFireDataset(Classifications)/ForestFireDataset/train'
generated_dir = '/kaggle/input/forest-fire-classification-dataset/forest_fire_diffusion_generated/forest_fire_diffusion_generated'

original_images, original_labels = load_images_from_dir(original_dir)
generated_images, generated_labels = load_images_from_dir(generated_dir)

all_images = np.concatenate([original_images, generated_images], axis=0)
all_labels = np.concatenate([original_labels, generated_labels], axis=0)

all_images = all_images.astype('float32') / 255.0


In [4]:
print(f"Total images: {len(all_images)}, Total labels: {len(all_labels)}")

unique, counts = np.unique(all_labels, return_counts=True)
print(f"All labels distribution: {dict(zip(unique, counts))}")

Total images: 5148, Total labels: 5148
All labels distribution: {0: 2456, 1: 2692}


In [5]:
import numpy as np
import os
import time
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import (confusion_matrix, precision_score, recall_score, 
                             f1_score, jaccard_score, matthews_corrcoef)
from sklearn.model_selection import KFold

def create_model_cnn(learning_rate=0.0001):
    input_layer = Input(shape=(150, 150, 3))
    x = Conv2D(32, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(512, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold_train_loss = []
fold_val_loss = []
fold_train_acc = []
fold_val_acc = []
confusion_matrices = []
precision_list = []
recall_list = []
f1_list = []
jaccard_list = []
mcc_list = []
model_sizes = []
train_times = []
inference_times = []

all_labels = np.array(all_labels)
all_images = np.array(all_images)

fold_num = 1
for train_index, val_index in kf.split(all_images):
    print(f"Fold {fold_num}")
    X_train, X_val = all_images[train_index], all_images[val_index]
    y_train, y_val = all_labels[train_index], all_labels[val_index]
    start_time = time.time()
    model = create_model_cnn(learning_rate=0.0001)
    model_creation_time = time.time() - start_time
    print(f"Model creation time: {model_creation_time:.2f} seconds")
    total_params = model.count_params()
    print(f"Number of Parameters in fold {fold_num}: {total_params}")
    model_file_path = f"model_fold_{fold_num}.h5"
    model.save(model_file_path)
    model_size = os.path.getsize(model_file_path) / (1024 * 1024)
    print(f"Model size for fold {fold_num}: {model_size:.2f} MB")
    model_sizes.append(model_size)
    os.remove(model_file_path)
    datagen = ImageDataGenerator(rotation_range=5, width_shift_range=0.1,
                                 height_shift_range=0.1, shear_range=0.1,
                                 zoom_range=0.1, horizontal_flip=True)
    datagen.fit(X_train)
    early_stopping = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)
    start_time = time.time()
    history = model.fit(datagen.flow(X_train, y_train, batch_size=32), 
                        validation_data=(X_val, y_val), 
                        epochs=50, 
                        callbacks=[early_stopping],
                        verbose=1)
    train_time = time.time() - start_time
    train_times.append(train_time)
    start_time = time.time()
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    inference_time = time.time() - start_time
    inference_times.append(inference_time)
    fold_train_loss.append(history.history['loss'])
    fold_val_loss.append(history.history['val_loss'])
    fold_train_acc.append(history.history['accuracy'])
    fold_val_acc.append(history.history['val_accuracy'])
    y_val_pred = model.predict(X_val, verbose=0)
    y_val_pred_class = np.where(y_val_pred > 0.5, 1, 0).flatten()
    confusion_matrices.append(confusion_matrix(y_val, y_val_pred_class))
    precision = precision_score(y_val, y_val_pred_class)
    recall = recall_score(y_val, y_val_pred_class)
    f1 = f1_score(y_val, y_val_pred_class)
    jaccard = jaccard_score(y_val, y_val_pred_class)
    mcc = matthews_corrcoef(y_val, y_val_pred_class)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    jaccard_list.append(jaccard)
    mcc_list.append(mcc)
    fold_num += 1

average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)
average_f1 = np.mean(f1_list)
average_jaccard = np.mean(jaccard_list)
average_mcc = np.mean(mcc_list)
average_model_size = np.mean(model_sizes)
average_train_time = np.mean(train_times)
average_inference_time = np.mean(inference_times)

average_train_loss = np.mean([loss[-1] for loss in fold_train_loss])
average_val_loss = np.mean([loss[-1] for loss in fold_val_loss])
average_train_acc = np.mean([acc[-1] for acc in fold_train_acc])
average_val_acc = np.mean([acc[-1] for acc in fold_val_acc])

print(f"\nAverage Training Loss: {average_train_loss:.4f}")
print(f"Average Validation Loss: {average_val_loss:.4f}")
print(f"Average Training Accuracy: {average_train_acc:.4f}")
print(f"Average Validation Accuracy: {average_val_acc:.4f}")
print(f"Average Precision: {average_precision:.4f}")
print(f"Average Recall: {average_recall:.4f}")
print(f"Average F1 Score: {average_f1:.4f}")
print(f"Average Jaccard Index: {average_jaccard:.4f}")
print(f"Average MCC: {average_mcc:.4f}")
print(f"Average Model Size: {average_model_size:.2f} MB")
print(f"Average Training Time: {average_train_time:.2f} seconds")
print(f"Average Inference Time: {average_inference_time:.2f} seconds")



Fold 1
Model creation time: 0.80 seconds
Number of Parameters in fold 1: 2753217
Model size for fold 1: 10.58 MB
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1731086963.812180     112 service.cc:145] XLA service 0x7b3424004410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731086963.812251     112 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  3/129 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - accuracy: 0.5920 - loss: 0.9784 

I0000 00:00:1731086971.639125     112 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


129/129 ━━━━━━━━━━━━━━━━━━━━ 39s 206ms/step - accuracy: 0.7695 - loss: 0.5702 - val_accuracy: 0.4951 - val_loss: 0.6682
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 141ms/step - accuracy: 0.8692 - loss: 0.3420 - val_accuracy: 0.7932 - val_loss: 0.4930
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 142ms/step - accuracy: 0.8854 - loss: 0.3042 - val_accuracy: 0.8592 - val_loss: 0.3436
Epoch 4/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - accuracy: 0.9001 - loss: 0.2675 - val_accuracy: 0.9136 - val_loss: 0.2149
Epoch 5/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 142ms/step - accuracy: 0.9342 - loss: 0.2031 - val_accuracy: 0.9029 - val_loss: 0.2528
Epoch 6/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - accuracy: 0.9324 - loss: 0.1819 - val_accuracy: 0.9466 - val_loss: 0.1481
Epoch 7/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - accuracy: 0.9313 - loss: 0.1930 - val_accuracy: 0.9515 - val_loss: 0.1421
Epoch 8/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 141ms/step - accuracy: 0.9361 - loss: 0.1876 - val

...

129/129 ━━━━━━━━━━━━━━━━━━━━ 33s 185ms/step - accuracy: 0.7667 - loss: 0.5597 - val_accuracy: 0.4534 - val_loss: 0.7615
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 142ms/step - accuracy: 0.8781 - loss: 0.3208 - val_accuracy: 0.6563 - val_loss: 0.6089
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 139ms/step - accuracy: 0.9118 - loss: 0.2527 - val_accuracy: 0.8126 - val_loss: 0.4423
Epoch 4/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - accuracy: 0.9131 - loss: 0.2638 - val_accuracy: 0.8660 - val_loss: 0.3435
Epoch 5/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - accuracy: 0.9204 - loss: 0.2311 - val_accuracy: 0.9282 - val_loss: 0.1994
Epoch 6/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 138ms/step - accuracy: 0.9326 - loss: 0.1975 - val_accuracy: 0.9534 - val_loss: 0.1426
Epoch 7/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 141ms/step - accuracy: 0.9394 - loss: 0.1763 - val_accuracy: 0.9388 - val_loss: 0.1707
Epoch 8/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 19s 140ms/step - accuracy: 0.9346 - loss: 0.1901 - val

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


129/129 ━━━━━━━━━━━━━━━━━━━━ 38s 214ms/step - accuracy: 0.7621 - loss: 0.6026 - val_accuracy: 0.5044 - val_loss: 0.6740
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 148ms/step - accuracy: 0.8761 - loss: 0.3449 - val_accuracy: 0.8533 - val_loss: 0.4932
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 150ms/step - accuracy: 0.9025 - loss: 0.2835 - val_accuracy: 0.7891 - val_loss: 0.4110
Epoch 4/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 151ms/step - accuracy: 0.9099 - loss: 0.2627 - val_accuracy: 0.9242 - val_loss: 0.2106
Epoch 5/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.9175 - loss: 0.2278 - val_accuracy: 0.9427 - val_loss: 0.1884
Epoch 6/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 152ms/step - accuracy: 0.9197 - loss: 0.2205 - val_accuracy: 0.9524 - val_loss: 0.1811
Epoch 7/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 152ms/step - accuracy: 0.9256 - loss: 0.1920 - val_accuracy: 0.9466 - val_loss: 0.1762
Epoch 8/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 150ms/step - accuracy: 0.9459 - loss: 0.1553 - val

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


129/129 ━━━━━━━━━━━━━━━━━━━━ 35s 198ms/step - accuracy: 0.7591 - loss: 0.5782 - val_accuracy: 0.4655 - val_loss: 0.8402
Epoch 2/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.8544 - loss: 0.3800 - val_accuracy: 0.6239 - val_loss: 0.5884
Epoch 3/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.8841 - loss: 0.3083 - val_accuracy: 0.5607 - val_loss: 0.9409
Epoch 4/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 152ms/step - accuracy: 0.9138 - loss: 0.2373 - val_accuracy: 0.8814 - val_loss: 0.3242
Epoch 5/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 155ms/step - accuracy: 0.9110 - loss: 0.2401 - val_accuracy: 0.8824 - val_loss: 0.3053
Epoch 6/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 153ms/step - accuracy: 0.9269 - loss: 0.1907 - val_accuracy: 0.9339 - val_loss: 0.2169
Epoch 7/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 152ms/step - accuracy: 0.9325 - loss: 0.1768 - val_accuracy: 0.9475 - val_loss: 0.1785
Epoch 8/50
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 152ms/step - accuracy: 0.9395 - loss: 0.1739 - val